In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.<INSERT_VERSION>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,929 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-19 13:15:43--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.30MB/s    in 0.7s    

2021-02-19 13:15:45 (1.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Michelin Star Dining").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/unit-7/michelin_star_dining.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("michelin_star_dining.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+----+---------+-----------+-------------------+----------+-------+---------------+-----+--------------------+
|                name|year| latitude|  longitude|               city|    region|zipCode|        cuisine|price|                 url|
+--------------------+----+---------+-----------+-------------------+----------+-------+---------------+-----+--------------------+
|        Kilian Stuba|2019| 47.34858|   10.17114|     Kleinwalsertal|   Austria|  87568|       Creative|$$$$$|https://guide.mic...|
|       Pfefferschiff|2019| 47.83787|   13.07917|           Hallwang|   Austria|   5300|Classic cuisine|$$$$$|https://guide.mic...|
|           Esszimmer|2019| 47.80685|   13.03409|           Salzburg|   Austria|   5020|       Creative|$$$$$|https://guide.mic...|
|          Carpe Diem|2019| 47.80001|   13.04006|           Salzburg|   Austria|   5020| Market cuisine|$$$$$|https://guide.mic...|
|              Edvard|2019|48.216503|   16.36852|               Wien|   Aust

In [ ]:
postgres_url="jdbc:postgresql://<Database URL>:5432/<Database Name>"
config = {"user":"postgres", 
          "password": "<PASSWORD>", 
          "driver":"org.postgresql.Driver"}
df.write.jdbc(postgres_url,table="restaurantweek",mode="append", properties=config)